# 기본 LSTM 코드

In [46]:
import pandas as pd
import numpy as np
import random
import os

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader 

from tqdm.auto import tqdm

In [47]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42) # Seed 고정

In [48]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')
sample_submission = pd.read_csv('./sample_submission.csv')

In [49]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204000 entries, 0 to 203999
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   num_date_time  204000 non-null  object 
 1   건물번호           204000 non-null  int64  
 2   일시             204000 non-null  object 
 3   기온(C)          204000 non-null  float64
 4   강수량(mm)        43931 non-null   float64
 5   풍속(m/s)        203981 non-null  float64
 6   습도(%)          203991 non-null  float64
 7   일조(hr)         128818 non-null  float64
 8   일사(MJ/m2)      116087 non-null  float64
 9   전력소비량(kWh)     204000 non-null  float64
dtypes: float64(7), int64(1), object(2)
memory usage: 15.6+ MB


In [50]:
# 일조, 일사 열 제거
train_df = train_df.drop(['일조(hr)','일사(MJ/m2)'], axis=1)
train_df.head()


,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),전력소비량(kWh)
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,1085.28
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,1047.36
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,974.88
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,953.76
4,1_20220601 04,1,20220601 04,18.4,NaN,2.8,43.0,986.40


In [51]:
train_df.isna().sum()

num_date_time         0
건물번호                  0
일시                    0
기온(C)                 0
강수량(mm)          160069
풍속(m/s)              19
습도(%)                 9
전력소비량(kWh)            0
dtype: int64

In [52]:
# 강수량 결측치 0.0으로 채우기
train_df['강수량(mm)'].fillna(0.0, inplace=True)

# 풍속, 습도 결측치 평균으로 채우고 반올림하기
train_df['풍속(m/s)'].fillna(round(train_df['풍속(m/s)'].mean(),2), inplace=True)
train_df['습도(%)'].fillna(round(train_df['습도(%)'].mean(),2), inplace=True)

In [53]:
train_df['month'] = train_df['일시'].apply(lambda x : float(x[4:6]))
train_df['day'] = train_df['일시'].apply(lambda x : float(x[6:8]))
train_df['time'] = train_df['일시'].apply(lambda x : float(x[9:11]))

In [97]:
# 순서 재배치
train_df = train_df[train_df.columns[:7].to_list() + train_df.columns[8:].to_list() + train_df.columns[7:8].to_list()]
train_df.head()

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),day,time,전력소비량(kWh),month
0,1_20220601 00,1,20220601 00,18.6,0.0,0.9,42.0,1.0,0.0,1085.28,6.0
1,1_20220601 01,1,20220601 01,18.0,0.0,1.1,45.0,1.0,1.0,1047.36,6.0
2,1_20220601 02,1,20220601 02,17.7,0.0,1.5,45.0,1.0,2.0,974.88,6.0
3,1_20220601 03,1,20220601 03,16.7,0.0,1.4,48.0,1.0,3.0,953.76,6.0
4,1_20220601 04,1,20220601 04,18.4,0.0,2.8,43.0,1.0,4.0,986.40,6.0


In [98]:
df = train_df.copy()
#df

In [99]:
df = train_df.copy()
df.rename(columns={'num_date_time': 'date_time'}, inplace=True)


In [100]:

df['date_time'] = df['date_time'].str.replace('.*_', '', regex=True)


In [101]:
df

,date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),day,time,전력소비량(kWh),month
0,20220601 00,1,20220601 00,18.6,0.0,0.9,42.0,1.0,0.0,1085.28,6.0
1,20220601 01,1,20220601 01,18.0,0.0,1.1,45.0,1.0,1.0,1047.36,6.0
2,20220601 02,1,20220601 02,17.7,0.0,1.5,45.0,1.0,2.0,974.88,6.0
3,20220601 03,1,20220601 03,16.7,0.0,1.4,48.0,1.0,3.0,953.76,6.0
4,20220601 04,1,20220601 04,18.4,0.0,2.8,43.0,1.0,4.0,986.40,6.0
...,...,...,...,...,...,...,...,...,...,...,...
203995,20220824 19,100,20220824 19,23.1,0.0,0.9,86.0,24.0,19.0,881.04,8.0
203996,20220824 20,100,20220824 20,22.4,0.0,1.3,86.0,24.0,20.0,798.96,8.0
203997,20220824 21,100,20220824 21,21.3,0.0,1.0,92.0,24.0,21.0,825.12,8.0
203998,20220824 22,100,20220824 22,21.0,0.0,0.3,94.0,24.0,22.0,640.08,8.0


In [103]:
# 데이터 변환

df['date_time'] = pd.to_datetime(df['date_time'], format='%Y%m%d %H')


In [104]:


#df['date_time'] = pd.to_datetime(df['date_time'], format='%Y%m%d %H')  # 년월일 시간 형태로 변환
df['date_time'] = df['date_time'].dt.strftime('%Y-%m-%d %H:%M:%S')  # 원하는 형식으로 포맷

print(df)

                  date_time  건물번호           일시  기온(C)  강수량(mm)  풍속(m/s)  \
0       2022-06-01 00:00:00     1  20220601 00   18.6      0.0      0.9   
1       2022-06-01 01:00:00     1  20220601 01   18.0      0.0      1.1   
2       2022-06-01 02:00:00     1  20220601 02   17.7      0.0      1.5   
3       2022-06-01 03:00:00     1  20220601 03   16.7      0.0      1.4   
4       2022-06-01 04:00:00     1  20220601 04   18.4      0.0      2.8   
...                     ...   ...          ...    ...      ...      ...   
203995  2022-08-24 19:00:00   100  20220824 19   23.1      0.0      0.9   
203996  2022-08-24 20:00:00   100  20220824 20   22.4      0.0      1.3   
203997  2022-08-24 21:00:00   100  20220824 21   21.3      0.0      1.0   
203998  2022-08-24 22:00:00   100  20220824 22   21.0      0.0      0.3   
203999  2022-08-24 23:00:00   100  20220824 23   20.7      0.0      0.1   

        습도(%)   day  time  전력소비량(kWh)  month  
0        42.0   1.0   0.0     1085.28    6.0  
1    

In [114]:
df.set_index('date_time', inplace=True)  # 'date_time' 열을 인덱스로 설정

In [115]:
df

,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),day,time,전력소비량(kWh),month
date_time,,,,,,,,,,
2022-06-01 00:00:00,1,20220601 00,18.6,0.0,0.9,42.0,1.0,0.0,1085.28,6.0
2022-06-01 01:00:00,1,20220601 01,18.0,0.0,1.1,45.0,1.0,1.0,1047.36,6.0
2022-06-01 02:00:00,1,20220601 02,17.7,0.0,1.5,45.0,1.0,2.0,974.88,6.0
2022-06-01 03:00:00,1,20220601 03,16.7,0.0,1.4,48.0,1.0,3.0,953.76,6.0
2022-06-01 04:00:00,1,20220601 04,18.4,0.0,2.8,43.0,1.0,4.0,986.40,6.0
...,...,...,...,...,...,...,...,...,...,...
2022-08-24 19:00:00,100,20220824 19,23.1,0.0,0.9,86.0,24.0,19.0,881.04,8.0
2022-08-24 20:00:00,100,20220824 20,22.4,0.0,1.3,86.0,24.0,20.0,798.96,8.0
2022-08-24 21:00:00,100,20220824 21,21.3,0.0,1.0,92.0,24.0,21.0,825.12,8.0


In [123]:
test_df.rename(columns={'num_date_time': 'date_time'}, inplace=True)
test_df['date_time'] = test_df['date_time'].str.replace('.*_', '', regex=True)
test_df['date_time'] = pd.to_datetime(test_df['date_time'], format='%Y%m%d %H')
test_df['date_time'] = test_df['date_time'].dt.strftime('%Y-%m-%d %H:%M:%S')
#test_df.set_index('date_time', inplace=True)
print(test_df)

                     건물번호           일시  기온(C)  강수량(mm)  풍속(m/s)  습도(%)
date_time                                                             
2022-08-25 00:00:00     1  20220825 00   23.5      0.0      2.2     72
2022-08-25 01:00:00     1  20220825 01   23.0      0.0      0.9     72
2022-08-25 02:00:00     1  20220825 02   22.7      0.0      1.5     75
2022-08-25 03:00:00     1  20220825 03   22.1      0.0      1.3     78
2022-08-25 04:00:00     1  20220825 04   21.8      0.0      1.0     77
...                   ...          ...    ...      ...      ...    ...
2022-08-31 19:00:00   100  20220831 19   22.5      0.0      0.9     84
2022-08-31 20:00:00   100  20220831 20   20.7      0.0      0.4     95
2022-08-31 21:00:00   100  20220831 21   20.2      0.0      0.4     98
2022-08-31 22:00:00   100  20220831 22   20.1      0.0      1.1     97
2022-08-31 23:00:00   100  20220831 23   19.1      0.0      0.1     99

[16800 rows x 6 columns]


In [124]:
from epftoolbox.data import read_data
from epftoolbox.models import DNN, evaluate_dnn_in_test_dataset
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

model = DNN(
    experiment_id=1,
    path_hyperparameter_folder="../elect/epftoolbox/examples/experimental_files/", #기본 "..\\..\\epftoolbox\\examples\\experimental_files\\" ../../epftoolbox/examples/experimental_files/
    nlayers=2,
    dataset="PJM",
    years_test=2,
    shuffle_train=True,
    data_augmentation=0,
    calibration_window=4,
)


In [133]:
test_df.index[1000]

'2022-08-31 16:00:00'

In [138]:
model

In [135]:
model.predict(test_df.loc[test_df.index[1000] - pd.Timedelta(weeks=2):, :])


AttributeError: 'DNN' object has no attribute 'model'

In [ ]:
model.recalibrate_and_forecast_next_day(df=data_available, next_day_date='27/12/2016')
pjm_test.index[0]
evaluate_dnn_in_test_dataset(
    1,
    path_hyperparameter_folder="..\\..\\epftoolbox\\examples\\experimental_files",
    path_datasets_folder="..\\..\\data\\epftoolbox\\PJM",
    shuffle_train=True,
    path_recalibration_folder="..\\..\\epftoolbox\\examples\\experimental_files",
    new_recalibration=True,
    begin_test_date="27/12/2016",
    end_test_date="27/12/2017",
)

In [10]:
# 하이퍼파라미터
input_size = 8  # feature의 개수
hidden_size = 64
num_layers = 2
output_size = 1
num_epochs = 5
window_size = 24  # 예측에 사용될 시간 윈도우 크기
batch_size = 64
learning_rate = 0.001

In [11]:
class TimeSeriesDataset(Dataset):
    def __init__(self, df, window_size):
        self.df = df
        self.window_size = window_size
    def __len__(self):
        return len(self.df) - self.window_size

    def __getitem__(self, idx):
        x = torch.tensor(self.df[idx:idx+self.window_size, :], dtype=torch.float)
        if self.df.shape[1] > 1:
            y = torch.tensor(self.df[idx+self.window_size, -1], dtype=torch.float)
        else:
            y = None
        return x, y

def create_data_loader(df, window_size, batch_size):
    dataset = TimeSeriesDataset(df, window_size)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    return data_loader

In [13]:
# normalization
scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_df.drop(['num_date_time', '건물번호', '일시'], axis=1).values)
train_loader = create_data_loader(train_data, window_size, batch_size)

In [17]:
k = TimeSeriesDataset(train_data, window_size)

In [26]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])

        return out

In [43]:
k.__getitem__(0)[0].size(0)

24

In [44]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"current device: {device}")

model = LSTM(input_size, hidden_size, num_layers, output_size).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

current device: cuda


In [45]:
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.unsqueeze(1).to(device)

        # Forward
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 300 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.6f}' 
                   .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))

Epoch [1/5], Step [300/3188], Loss: 0.000448
Epoch [1/5], Step [600/3188], Loss: 0.000652
Epoch [1/5], Step [900/3188], Loss: 0.000142
Epoch [1/5], Step [1200/3188], Loss: 0.001297
Epoch [1/5], Step [1500/3188], Loss: 0.017534
Epoch [1/5], Step [1800/3188], Loss: 0.003190
Epoch [1/5], Step [2100/3188], Loss: 0.000030
Epoch [1/5], Step [2400/3188], Loss: 0.000093
Epoch [1/5], Step [2700/3188], Loss: 0.000112
Epoch [1/5], Step [3000/3188], Loss: 0.002827
Epoch [2/5], Step [300/3188], Loss: 0.001489
Epoch [2/5], Step [600/3188], Loss: 0.000535
Epoch [2/5], Step [900/3188], Loss: 0.000209
Epoch [2/5], Step [1200/3188], Loss: 0.001114
Epoch [2/5], Step [1500/3188], Loss: 0.009415
Epoch [2/5], Step [1800/3188], Loss: 0.000294
Epoch [2/5], Step [2100/3188], Loss: 0.000030
Epoch [2/5], Step [2400/3188], Loss: 0.000186
Epoch [2/5], Step [2700/3188], Loss: 0.000098
Epoch [2/5], Step [3000/3188], Loss: 0.000817
Epoch [3/5], Step [300/3188], Loss: 0.000364
Epoch [3/5], Step [600/3188], Loss: 0.000

In [ ]:
# 학습 데이터에서 마지막 행 가져오기
last_train_data = train_df.drop(['num_date_time', '건물번호', '일시',], axis=1).loc[204000-24:,:]

# 실수형 데이터로 변환
test_df['습도(%)'] = test_df['습도(%)'].astype('float64')

# 날짜 데이터 추가
test_df['month'] = test_df['일시'].apply(lambda x : float(x[4:6]))
test_df['day'] = test_df['일시'].apply(lambda x : float(x[6:8]))
test_df['time'] = test_df['일시'].apply(lambda x : float(x[9:11]))

# 전력소비량 열 생성
final_df = pd.concat((test_df.drop(['num_date_time', '건물번호', '일시',], axis=1), pd.DataFrame(np.zeros(test_df.shape[0]))),axis=1)
final_df = final_df.rename({0:'전력소비량(kWh)'},axis=1)

In [50]:
test_df = pd.concat((last_train_data, final_df)).reset_index(drop=True)
test_data = scaler.transform(test_df.values) # train과 동일하게 scaling
test_data.shape

(16824, 8)

In [51]:
# Dataset & DataLoader
test_dataset = TimeSeriesDataset(test_data, window_size)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [52]:
model.eval()

test_predictions = []

with torch.no_grad():
    for i in range(test_data.shape[0] - window_size):
        x = torch.Tensor(test_data[i:i+window_size,:]).to(device)
        new_x = model(x.view(1,window_size,-1))
        
        test_data[i+window_size,-1] = new_x # 입력 업데이트
        test_predictions.append(new_x.detach().cpu().numpy().item()) # 예측 결과 저장

In [53]:
predictions = scaler.inverse_transform(test_data)[24:,-1] # 원래 scale로 복구

In [56]:
sample_submission['answer'] = predictions
sample_submission.head()

,num_date_time,answer
0,1_20220825 00,488.865039
1,1_20220825 01,520.486610
2,1_20220825 02,555.244625
3,1_20220825 03,602.879989
4,1_20220825 04,680.498247


In [57]:
sample_submission.to_csv('lstm_baseline_submission.csv', index=False)